In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow import keras
import zipfile
from PIL import Image
import cv2

# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as vis_util

os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.insert(0, "../")

from mystique.initial_setups import set_graph_and_tensors
from mystique.predict_card import PredictCard
from mystique.detect_objects import ObjectDetection
from mystique.utils import plot_results


# This is needed to display the images.
%matplotlib inline

## Frozen_graph from saved_model

Using the tf2 object detection tools convert the checkpoint (The one works well) to a saved_model format.

from this saved_model format, using the below method convert it into a single frozen_graph for easy packaging with application in case of pic2card. There is no dedicated tools available to do this process in tf2 version as it got
depricated in tf2 version.

In [ ]:
graph_def = tf.compat.v1.GraphDef()
imported = tf.saved_model.load(
    "/home/haridas/projects/opensource/tf-models/research/object_detection/pic2card_model/saved_model/"
)

In [ ]:
concrete_fun = imported.signatures['serving_default']
frozen_func = convert_variables_to_constants_v2(concrete_fun)

In [ ]:
# convert_variables_to_constants_v2?

In [ ]:
frozen_func.graph.as_graph_def()

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="/home/haridas/projects/opensource/tf-models/research/object_detection/pic2card_model/",
                  name="frozen_graph1.pb",
                  as_text=False
                 )

In [ ]:
frozen_func.inputs

In [ ]:
output_nodes = [i.name for i in frozen_func.outputs]

In [ ]:
# layers = frozen_func.graph.get_operations()

In [ ]:
frozen_graph = "/home/haridas/projects/opensource/tf-models/research/object_detection/pic2card_model/frozen_graph.pb"

## Model Inference APIs

In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

#     if print_graph == True:
#         print("-" * 50)
#         print("Frozen model layers: ")
#         layers = [op.name for op in import_graph.get_operations()]
#         for layer in layers:
#             print(layer)
#         print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

In [ ]:
with tf.io.gfile.GFile(frozen_graph, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

In [ ]:
# output_nodes

In [ ]:
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["input_tensor:0"],
                                outputs=["Identity_1:0", "Identity_2:0", "Identity_4:0"]
                               )

In [ ]:
[i.name for i in frozen_func.graph.get_operations()[-20:]]

In [ ]:
def set_graph_and_tensors(tensors=("detection_boxes", "detection_scores",
                                   "detection_classes")):
    """
    setting up tf graphs and tensors using the trained inference graph

    @param tensors: required tensors from inference graph

    :return: detection_graph, category_index, tensor_dict
    """
    tensor_dict = dict()
    detection_graph = tf.Graph()
    # setting up default graph with graphs from inference graph
    with detection_graph.as_default() as default_graph:
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.compat.v1.gfile.GFile(
            config.TF_FROZEN_MODEL_PATH, "rb"
        ) as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name="")
        ops = default_graph.get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        for tensor in tensors:
            tmp_tensor_name = tensor + ":0"
            if tmp_tensor_name in all_tensor_names:
                tensor_dict[tensor] = default_graph.get_tensor_by_name(
                    tmp_tensor_name
                )

    return detection_graph, tensor_dict


def run_inference_for_single_image(self, image: np.array):
    """
    Runs the inference graph for the given image
    @param image: numpy array of input design image
    @return: output dict of objects, classes and coordinates
    """
    # Run inference
    detection_graph = self.detection_graph
    with detection_graph.as_default():
        image_tensor = detection_graph.get_tensor_by_name(
            "image_tensor:0")
        with tf.compat.v1.Session() as sess:
            output_dict = sess.run(
                self.tensor_dict, feed_dict={
                    image_tensor: np.expand_dims(
                        image, 0)})

    # all outputs are float32 numpy arrays, so convert types as
    # appropriate
    output_dict["detection_classes"] = output_dict[
        "detection_classes"][0].astype(np.uint8)
    output_dict["detection_boxes"] = output_dict[
        "detection_boxes"][0]
    output_dict["detection_scores"] = output_dict[
        "detection_scores"][0]

    return output_dict

In [ ]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/3.png"
img = Image.open(img_path)
img_np = np.asarray(img)
img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

In [ ]:
img

In [ ]:
# keras.datasets.fashion_mnist.load_data?

In [ ]:
tf.expand_dims(tf.constant(img_np), axis=0)

In [ ]:
bboxes, labels, scores = frozen_func(input_tensor=tf.expand_dims(tf.constant(img_np), axis=0))

In [ ]:
# tensor_outputs[4]


In [ ]:
def predict_bbox(img_path) -> np.array:
    """
    Predict the bounding boxes, class label and draw the bbox
    on the original image.
    """
    img = Image.open(img_path)
    img_np = np.asarray(img)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    #img_np_copy = img_np.copy()
    object_detection = ObjectDetection()
    
    output = object_detection.get_objects(img_np, img)
    # google models wants ymin, xmin, ymax, xmax format.
    # output["detection_boxes"] = output["detection_boxes"][:, [1, 0, 3, 2]]
    
    # print(output.keys())
    
    _img = plot_results(img,
                 output["detection_classes"],
                 output["detection_scores"],
                 output["detection_boxes"]
                )
    return _img

In [ ]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/3.png"

Image.open(predict_bbox(img_path))

In [ ]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/1.png"
Image.open(predict_bbox(img_path))

## Model Graph Analysis

In [ ]:
graph, gconf = set_graph_and_tensors()

In [ ]:
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

## RCNN and Family

Inspect the different aspect of the RCNN family of models and debug and tune them based on the
necessity.

### Anchor box generation example

In [ ]:
model_dir = "/home/haridas/projects/AdaptiveCards/source/pic2card/out/frcnn-2020-07-05-1593958532/"

In [ ]:
last_checkpoint = tf.train.latest_checkpoint(model_dir)